In [ ]:
import numpy
from collections import deque
import pandas
import math
import pandas_datareader.data as web
import datetime
import requests
import requests_cache
import xlrd
import tempfile
import itertools

In [ ]:
def get_japan_rates():
    url = 'http://www.mof.go.jp/english/jgbs/reference/interest_rate/historical/jgbcme_all.csv'
    expire_after = datetime.timedelta(days=3)
    session = requests_cache.CachedSession(cache_name='data-cache', backend='sqlite', expire_after=expire_after)

    r = session.get(url, stream=True)

    with tempfile.NamedTemporaryFile(suffix='.csv') as tmp:
        for chunk in r.iter_content(chunk_size=1024):
            tmp.write(chunk)
            
        tmp.flush()
        
        df = pandas.read_csv(tmp.name, header=1, index_col=0, parse_dates=True, na_values='-')


    # The MOF data is daily. Which is overkill. We just want to rate at the
    # start of each month.
    df = df.groupby(by=lambda x: datetime.date(x.year, x.month, 1)).first()

    # The MOF data reports rates at 7.98 instead of 0.0798, which is what we want
    df = df.applymap(lambda x: x/100)
    return df

all_by_month = get_japan_rates()
all_by_month.head()

In [ ]:
def iterate_fund(ladder, yield_curve, max_maturity):
    ladder.reduce_maturities()
    ladder.generate_payments()
#    print(ladder.cash)
#    import pdb;pdb.set_trace()
    sold_bonds = ladder.sell_bonds(yield_curve)

    # Only buy a new bond if we actually sold one...
    if sold_bonds:
        ladder.buy_bond(yield_curve[max_maturity-1], max_maturity)
    
    # This happens *after* we sell the shortest bond and buy a new long one
    # (at least, that's what longinvest %doctest_modees...)
    nav = ladder.get_nav(yield_curve)

    return (ladder, nav)

In [ ]:
def a2m(annual_rate):
    return pow(annual_rate + 1, 1/12) - 1

class Bond:
    def __init__(self, face_value, yield_pct, maturity, payments_per_year=12):
        self.face_value = face_value
        self.yield_pct = yield_pct
        self.maturity = maturity
        self.payments_per_year = payments_per_year
        
    def __repr__(self):
        return ('Maturity: %d | Yield: %.2f%% | Face Value: $%.2f' % (self.maturity, self.yield_pct * 100, self.face_value))

    def gen_payment(self):
        return self.face_value * self.yield_pct / self.payments_per_year
    
    def value(self, rates):
        value = numpy.pv(rates[self.maturity - 1], self.maturity / 12, (self.face_value * self.yield_pct), self.face_value)
        return -value
    
class BondLadder:
    def __init__(self, min_maturity, max_maturity):
        self.min_maturity = min_maturity
        self.max_maturity = max_maturity
        self.cash = 0
        
        self.ladder = set()
        
    def print_all(self):
        for bond in sorted(self.ladder, key=lambda b: b.maturity):
            print(bond)
            
    def print_all_values(self, rates):
        for bond in sorted(self.ladder, key=lambda b: b.maturity):
            print(bond.value(rates))
        
    def buy_bond(self, rate, maturity):
        b = Bond(self.cash, rate, maturity)
        self.add_bond(b)
        self.cash = 0
        return b
        
    def get_nav(self, rates):
        return self.cash + sum((b.value(rates) for b in self.ladder))

    def generate_payments(self):
        self.cash += sum((b.gen_payment() for b in self.ladder))        
        
    def __repr__(self):
        return ('%d-%d Ladder { Num Bonds: %d. }' % (self.max_maturity, self.min_maturity, len(self.ladder)))
        
    def add_bond(self, bond):
        #assert bond.maturity <= self.max_maturity
        #assert bond.maturity >= self.min_maturity
        self.ladder.add(bond)
    
    def reduce_maturities(self):
        for bond in self.ladder:
            bond.maturity -= 1

    def sell_bonds(self, rates):
        to_sell = filter(lambda bond: bond.maturity <= self.min_maturity, self.ladder)
        to_sell = list(to_sell)
        self.ladder = self.ladder.difference(to_sell)
        self.cash += sum((b.value(rates) for b in to_sell))
        return to_sell

In [ ]:
def bootstrap(yield_curve, max_bonds, min_maturity):
    bond_yield = yield_curve[max_bonds - 1]

    # Why - 11?
    #min_maturity -= 11

    ladder = BondLadder(min_maturity, max_bonds)
    starting_face_value = 50 # chosen arbitrarily (to match longinvest)

    for i, j in zip(range(max_bonds), range(min_maturity, max_bonds+1)):
        face_value = pow(1 + a2m(bond_yield), i) * starting_face_value
        b = Bond(face_value, bond_yield, j)
        ladder.add_bond(b)
    return ladder
bootstrap([.0532]*120, 10*12, 5*12)

In [ ]:
def splice_data(raw_rates, series):
    # Start by loading the data we get from Shiller.
    # This will always exist.

    def safe_add(series_index, rate_index):
        # Don't overwrite any data we already have.
        if math.isnan(series.iloc[series_index]):
            series.iloc[series_index] = raw_rates[rate_index]

    safe_add(1 * 12 - 1, '1Y')
    safe_add(2 * 12 - 1, '2Y')
    safe_add(3 * 12 - 1, '3Y')
    safe_add(4 * 12 - 1, '4Y')
    safe_add(5 * 12 - 1, '5Y')
    safe_add(6 * 12 - 1, '6Y')
    safe_add(7 * 12 - 1, '7Y')
    safe_add(8 * 12 - 1, '8Y')
    safe_add(9 * 12 - 1, '9Y')
    safe_add(10 * 12 - 1, '10Y')
         

def build_yield_curve(raw_rates, yield_curve_size=10*12):
    s = pandas.Series(math.nan, index=numpy.arange(yield_curve_size))

    # We use NaN to indicate "the data needs to be interpolated"
    # We have a few different data series that we splice together.
    splice_data(raw_rates, s)
    
    # This will do linear interpolation where it can.
    s.interpolate(inplace=True)
    
    # But it can still leave us with NaNs at the low end of the range
    s.fillna(method='backfill', inplace=True)
    
    return s.tolist()

In [ ]:
['%.3f' % (s*100) for s in build_yield_curve(all_by_month.iloc[0])]

In [ ]:
bootstrap(build_yield_curve(all_by_month.iloc[-2]), 10 * 12, 4 * 12)

In [ ]:
def loop(ladder, rates, max_maturity):
    df = pandas.DataFrame(columns=['NAV', 'Change'])

    # The first iterations have fake data with duplicate years
    # But that's okay because we overwrite them with later data
    # (since they all have the same year)
    for (year, current_rates) in rates:
        if year.year % 5 == 0 and year.month == 1:
            print('Calculating...', year.year)
        (ladder, nav) = iterate_fund(ladder, build_yield_curve(current_rates), max_maturity)
        df.loc[year] = {'NAV' : nav, 'Change' : None}

    calculate_returns(df)
    return df

def calculate_returns(df):
    # Longinvest calculates the return based on comparison's to
    # next year's NAV. So I'll do the same. Even though that seems
    # weird to me. Maybe it's because the rates are based on January?
    # Hmmm...that sounds plausible.
    max_row = df.shape[0]

    for i in range(max_row - 1):
        next_nav = df.iloc[i+1]['NAV']
        nav = df.iloc[i]['NAV']
        change = (next_nav - nav) / nav
        df.iloc[i]['Change'] = change
    return df

def make_annual_ladder(max_maturity, min_maturity, yields):
    rate = yields[max_maturity - 1]
    
    # We have to add the "- 12" in order to make things like up with how
    # longinvest runs things. His "10-4" ladder is really more of "10-3" ladder:
    # bonds get sold the moment they become a 3 year bond.
    ladder = BondLadder(min_maturity - 12, max_maturity)

    face_value = 50
    for i in range(min_maturity, max_maturity + 1, 12):
        ladder.add_bond(Bond(face_value, rate, i))
        face_value = face_value * (1 + rate)

    return ladder

def simulate_monthly_turnover(max_maturity, min_maturity, rates):
    min_maturity = min_maturity * 12
    max_maturity = max_maturity * 12

    initial_yields = build_yield_curve(rates.iloc[0])
    ladder = bootstrap(initial_yields, max_maturity, min_maturity)

    return loop(ladder, rates.iterrows(), max_maturity)

def simulate_annual_turnover(max_maturity, min_maturity, rates):
    min_maturity = min_maturity * 12
    max_maturity = max_maturity * 12

    initial_yields = build_yield_curve(rates.iloc[0])
    ladder = make_annual_ladder(max_maturity, min_maturity, initial_yields)

    # longinvest actually simulates 1870 and assumes 1871 rates. That's why,
    # when the simulation starts in January 1871, all the bonds have paid 1 year
    # of interest and one of the bonds is ready to be sold.
    # So we need to generate 11 months of fake data to do the same simulation.
    # Why 11 months? The 12th month is the real January 1871 data.
    first_index = rates.head(1).index
    bootstrap_rates = itertools.repeat(next(rates.iterrows()), 11)
    all_rates = itertools.chain(bootstrap_rates, rates.iterrows())

    return loop(ladder, all_rates, max_maturity)

In [ ]:
%%time
#sim_results = simulate_annual_turnover(10, 4, all_by_month)
sim_results = simulate_monthly_turnover(5, 1, all_by_month)
print(sim_results.head())

In [ ]:
sim_results.to_csv('bonds-monthly-japan.csv')

In [ ]:
def get_morningstar(secid):
    url = 'http://mschart.morningstar.com/chartweb/defaultChart?type=getcc&secids=%s&dataid=8225&startdate=1900-01-01&enddate=2016-11-18&currency=&format=1' % secid
    expire_after = datetime.timedelta(days=3)
    session = requests_cache.CachedSession(cache_name='data-cache', backend='sqlite', expire_after=expire_after)

    # TODO: why doesn't this work!?!
    r = session.get(url)
    #r = requests.get(url)
    j = r.json()
    
    # The Morningstar data is pretty deeply nested....
    m = j['data']['r'][0]
    assert m['i'] == secid
    
    actual_data = m['t'][0]['d']
    # convert from strings to real data types
    as_dict = dict([(datetime.datetime.strptime(n['i'], '%Y-%m-%d'), float(n['v'])) for n in m['t'][0]['d']])
    
    # Strip out data?
    # Do we only want start of month, end of month, start of year, end of year, etc?
    s = pandas.Series(as_dict, name=secid)

    return s

barclays_index = get_morningstar('XIUSA000MJ')

# Use only final value for each calendar year
def annual(series):
    return series.groupby(by=lambda x: x.year).last()
# Use only final value for each calendar month
def monthly(series):
    return series.groupby(by=lambda x: datetime.date(x.year, x.month, 1)).last()

In [ ]:
monthly_results = pandas.concat([sim_results, monthly(barclays_index)], axis=1, join='outer')

def calculate_change_prev(df, column):
    max_row = df.shape[0]
    
    series = pandas.Series()

    for i in range(max_row - 1):
        val = df.iloc[i][column]
        prev_val = df.iloc[i-1][column]
        change = (val - prev_val) / prev_val
        series.loc[df.iloc[i].name] = change
    return series

def calculate_change_next(df, column):
    max_row = df.shape[0]
    
    series = pandas.Series()

    for i in range(max_row - 1):
        val = df.iloc[i][column]
        next_val = df.iloc[i+1][column]
        change = (next_val - val) / val
        series.loc[df.iloc[i].name] = change
    return series

index_change = calculate_change_prev(monthly_results, 'XIUSA000MJ')
monthly_results = monthly_results.assign(index_change=index_change)
print(monthly_results[["Change", "index_change"]].corr())

In [ ]:
# This is correct when doing monthly calculations.
annual_results = monthly_results.groupby(by=lambda x: x.year).first()
s_nav = calculate_change_next(annual_results, 'NAV')
s_ind = calculate_change_next(annual_results, 'XIUSA000MJ')
annual_results = annual_results.assign(Change=s_nav, index_change=s_ind)
print(annual_results[["Change", "index_change"]].corr())
annual_results.to_csv('bonds-monthly-annual.csv')
print(annual_results.head())